<a href="https://colab.research.google.com/github/jamiestephens/tideprediction/blob/main/Tide_Prediction_Data_Visualizations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np
from bokeh.io import show
from bokeh.models import ColumnDataSource, Range1d
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import DatetimeTickFormatter, DatetimeAxis
from bokeh.models import HoverTool
from bokeh.models import FixedTicker
from bokeh.embed import components
from bokeh.embed import autoload_static
output_notebook()
import imageio.v2 as imageio
import matplotlib.pyplot as plt
from datetime import datetime
import datetime

Mounted at /content/drive


# Data Load and Initial Transformation

In [ ]:
X_train = np.load('drive/MyDrive/source/X_train_surge_new.npz')
X_test = np.load('drive/MyDrive/source/X_test_surge_new.npz')
Y_train = pd.read_csv('drive/MyDrive/source/Y_train_surge.csv',index_col = 'id_sequence')

df_X_test = pd.DataFrame.from_dict({item: X_test[item] for item in X_test.files}, orient='index').T.set_index('id_sequence')
df_X_train = pd.DataFrame.from_dict({item: X_train[item] for item in X_train.files}, orient='index').T.set_index('id_sequence')

In [ ]:
def timestamp_to_datetime(timestamp):
    return pd.to_datetime('1970-01-01') + pd.to_timedelta(timestamp, unit='s')

def timestamps(df):
  column_list = ['t_slp','t_surge1_input','t_surge2_input','t_surge1_output','t_surge2_output']
  for col in column_list:
    df[col] = df[col].apply(lambda x: np.array([timestamp_to_datetime(t) for t in x]))
  return df


In [ ]:
df_X_test = timestamps(df_X_test)
df_X_train = timestamps(df_X_train)

# Input EDA

In [ ]:
df_X_train

,t_slp,slp,t_surge1_input,surge1_input,t_surge2_input,surge2_input,t_surge1_output,t_surge2_output
id_sequence,,,,,,,,
1,"[1950-01-01 20:59:44, 1950-01-02 00:00:00, 195...","[[[102389.25, 102458.25, 102514.25, 102554.25,...","[1950-01-01 12:00:00, 1950-01-02 00:00:00, 195...","[-0.54593754, 0.13110517, 0.31880018, 0.194787...","[1950-01-01 14:00:32, 1950-01-02 01:59:28, 195...","[-0.41432458, -1.0094675, -1.2022604, -1.19387...","[1950-01-06 16:00:00, 1950-01-07 04:00:00, 195...","[1950-01-06 16:59:44, 1950-01-07 04:59:44, 195..."
2,"[1950-01-03 00:00:00, 1950-01-03 03:00:16, 195...","[[[101895.6, 101970.6, 102032.6, 102077.6, 102...","[1950-01-02 12:59:44, 1950-01-03 00:59:44, 195...","[0.31880018, 0.19478741, -0.22752637, -0.40181...","[1950-01-02 14:00:32, 1950-01-03 03:00:16, 195...","[-1.2022604, -1.193878, -1.1435843, -0.8166747...","[1950-01-07 16:00:00, 1950-01-08 04:00:00, 195...","[1950-01-07 17:59:28, 1950-01-08 06:00:32, 195..."
3,"[1950-01-04 00:00:00, 1950-01-04 03:00:16, 195...","[[[101661.24, 101700.24, 101774.24, 101849.24,...","[1950-01-03 14:00:32, 1950-01-04 01:59:28, 195...","[-0.22752637, -0.4018146, -0.87440383, -0.6397...","[1950-01-03 15:00:16, 1950-01-04 03:00:16, 195...","[-1.1435843, -0.81667477, -0.7579987, -0.12932...","[1950-01-08 16:59:44, 1950-01-09 04:59:44, 195...","[1950-01-08 17:59:28, 1950-01-09 07:00:16, 195..."
4,"[1950-01-05 00:00:00, 1950-01-05 03:00:16, 195...","[[[101366.41, 101393.41, 101476.41, 101577.41,...","[1950-01-04 14:00:32, 1950-01-05 01:59:28, 195...","[-0.87440383, -0.63978505, -1.0755056, -1.7123...","[1950-01-04 16:00:00, 1950-01-05 04:00:00, 195...","[-0.7579987, -0.12932655, -0.17962031, -0.3724...","[1950-01-09 16:59:44, 1950-01-10 06:00:32, 195...","[1950-01-09 19:00:16, 1950-01-10 07:00:16, 195..."
5,"[1950-01-06 00:00:00, 1950-01-06 03:00:16, 195...","[[[101245.49, 101220.49, 101209.49, 101225.49,...","[1950-01-05 15:00:16, 1950-01-06 03:00:16, 195...","[-1.0755056, -1.712328, 0.58693594, 1.0695802,...","[1950-01-05 16:00:00, 1950-01-06 04:59:44, 195...","[-0.17962031, -0.3724131, -0.05388589, 0.35684...","[1950-01-10 17:59:28, 1950-01-11 06:00:32, 195...","[1950-01-10 20:00:00, 1950-01-11 08:00:00, 195..."
...,...,...,...,...,...,...,...,...
5595,"[2010-10-19 08:59:44, 2010-10-19 11:58:56, 201...","[[[101470.0, 101486.0, 101501.0, 101517.0, 101...","[2010-10-18 23:00:16, 2010-10-19 10:59:12, 201...","[1.9376696, 1.6092033, 0.86177504, 1.2974956, ...","[2010-10-19 00:59:44, 2010-10-19 13:00:48, 201...","[0.021554768, 0.62508005, 0.39037576, 0.457434...","[2010-10-24 01:59:28, 2010-10-24 15:00:16, 201...","[2010-10-24 03:58:56, 2010-10-24 16:00:00, 201..."
5596,"[2010-10-20 08:59:44, 2010-10-20 12:01:04, 201...","[[[101663.87, 101650.87, 101641.87, 101635.87,...","[2010-10-20 00:00:00, 2010-10-20 12:01:04, 201...","[0.86177504, 1.2974956, 0.50984687, 0.87518185...","[2010-10-20 01:59:28, 2010-10-20 14:00:32, 201...","[0.39037576, 0.45743412, 0.28140593, 0.4322872...","[2010-10-25 01:59:28, 2010-10-25 15:00:16, 201...","[2010-10-25 04:01:04, 2010-10-25 16:00:00, 201..."
5597,"[2010-10-21 08:59:44, 2010-10-21 11:58:56, 201...","[[[101876.15, 101838.15, 101801.15, 101765.15,...","[2010-10-21 00:00:00, 2010-10-21 13:00:48, 201...","[0.50984687, 0.87518185, 1.8505255, 1.193593, ...","[2010-10-21 01:59:28, 2010-10-21 14:00:32, 201...","[0.28140593, 0.43228725, 0.15567149, 0.2478767...","[2010-10-26 02:59:12, 2010-10-26 16:00:00, 201...","[2010-10-26 05:00:48, 2010-10-26 16:59:44, 201..."


In [ ]:
df_X_test

,t_slp,slp,t_surge1_input,surge1_input,t_surge2_input,surge2_input,t_surge1_output,t_surge2_output
id_sequence,,,,,,,,
5600,"[-629769600.0, -629758800.0, -629748000.0, -62...","[[[101557.15, 101724.15, 101877.15, 102005.15,...","[-629766000.0, -629722800.0, -629679600.0, -62...","[0.48638502, -1.3067727, -1.7324382, -1.367103...","[-629762400.0, -629715600.0, -629672400.0, -62...","[0.28140593, 0.6083154, -0.0371213, -0.5652059...","[-629323200.0, -629276400.0, -629233200.0, -62...","[-629316000.0, -629272800.0, -629226000.0, -62..."
5601,"[-627080400.0, -627069600.0, -627058800.0, -62...","[[[101447.93, 101333.93, 101233.93, 101155.93,...","[-627084000.0, -627037200.0, -626994000.0, -62...","[1.0729319, 0.6003427, 1.0561734, 0.3824824, 0...","[-627076800.0, -627033600.0, -626990400.0, -62...","[-0.49814755, -0.3724131, 0.08861313, -0.58197...","[-626641200.0, -626598000.0, -626554800.0, -62...","[-626637600.0, -626590800.0, -626547600.0, -62..."
5602,"[-619077600.0, -619066800.0, -619056000.0, -61...","[[[102084.63, 102055.63, 102022.63, 101983.63,...","[-619084800.0, -619041600.0, -618998400.0, -61...","[-0.84088683, 0.22830437, -0.26774675, -0.0398...","[-619081200.0, -619034400.0, -618991200.0, -61...","[0.73404986, 1.1280178, 0.22272985, 0.08861313...","[-618642000.0, -618591600.0, -618548400.0, -61...","[-618631200.0, -618584400.0, -618541200.0, -61..."
5603,"[-616798800.0, -616788000.0, -616777200.0, -61...","[[[102291.61, 102258.61, 102224.61, 102189.61,...","[-616802400.0, -616759200.0, -616712400.0, -61...","[0.06407124, 0.3255036, 0.12775348, 0.35566884...","[-616795200.0, -616752000.0, -616708800.0, -61...","[0.35684657, 0.58316857, 0.566404, 0.07184854,...","[-616359600.0, -616312800.0, -616266000.0, -61...","[-616352400.0, -616309200.0, -616262400.0, -61..."
5604,"[-614520000.0, -614509200.0, -614498400.0, -61...","[[[102255.68, 102238.68, 102220.68, 102200.68,...","[-614523600.0, -614476800.0, -614433600.0, -61...","[0.37913072, -0.1973611, -0.6900605, -0.887810...","[-614512800.0, -614469600.0, -614426400.0, -61...","[-0.120944254, -0.12932655, -0.045503594, -0.5...","[-614077200.0, -614030400.0, -613987200.0, -61...","[-614070000.0, -614026800.0, -613980000.0, -61..."
...,...,...,...,...,...,...,...,...
6104,"[1276171300.0, 1276182000.0, 1276192800.0, 127...","[[[102135.28, 102101.28, 102069.28, 102037.28,...","[1276171300.0, 1276214400.0, 1276261200.0, 127...","[-0.0934585, 0.12440178, 0.2752281, 1.3846397,...","[1276178400.0, 1276221600.0, 1276268400.0, 127...","[2.7122715, 2.385362, 2.0919816, 1.9159535, 1....","[1276614000.0, 1276660700.0, 1276704000.0, 127...","[1276624800.0, 1276668000.0, 1276711200.0, 127..."
6105,"[1278450000.0, 1278460800.0, 1278471600.0, 127...","[[[102328.64, 102302.64, 102271.64, 102234.64,...","[1278446300.0, 1278493200.0, 1278536400.0, 127...","[0.12440178, 0.3255036, 0.5601224, 0.9991946, ...","[1278457200.0, 1278504000.0, 1278547200.0, 127...","[0.4658164, 0.42390493, 0.50772786, 0.45743412...","[1278896400.0, 1278943200.0, 1278990000.0, 127...","[1278907100.0, 1278950400.0, 1278993700.0, 127..."
6106,"[1280728800.0, 1280739600.0, 1280750300.0, 128...","[[[102230.52, 102184.52, 102134.52, 102077.52,...","[1280728800.0, 1280772000.0, 1280815200.0, 128...","[0.057367843, 0.054016147, 0.11769839, 0.00709...","[1280732400.0, 1280779300.0, 1280822400.0, 128...","[1.0274302, 0.78434366, 0.9436072, 0.91846037,...","[1281178800.0, 1281225600.0, 1281268700.0, 128...","[1281186000.0, 1281232800.0, 1281276000.0, 128..."


In [ ]:
Y_train

,surge1_t0,surge1_t1,surge1_t2,surge1_t3,surge1_t4,surge1_t5,surge1_t6,surge1_t7,surge1_t8,surge1_t9,surge2_t0,surge2_t1,surge2_t2,surge2_t3,surge2_t4,surge2_t5,surge2_t6,surge2_t7,surge2_t8,surge2_t9
id_sequence,,,,,,,,,,,,,,,,,,,,
1,0.586936,1.069580,0.767928,-0.100162,0.070775,-0.244285,-0.354891,-0.928031,-0.773853,-0.375001,-0.053886,0.356847,0.348464,0.264641,0.901696,0.449052,0.113760,-0.422707,-0.456236,-0.825057
2,0.767928,-0.100162,0.070775,-0.244285,-0.354891,-0.928031,-0.773853,-0.375001,-0.361594,-0.210768,0.348464,0.264641,0.901696,0.449052,0.113760,-0.422707,-0.456236,-0.825057,-0.992703,-0.992703
3,0.070775,-0.244285,-0.354891,-0.928031,-0.773853,-0.375001,-0.361594,-0.210768,0.288635,-0.726929,0.901696,0.449052,0.113760,-0.422707,-0.456236,-0.825057,-0.992703,-0.992703,-0.322119,-0.883733
4,-0.354891,-0.928031,-0.773853,-0.375001,-0.361594,-0.210768,0.288635,-0.726929,-0.576103,1.160076,0.113760,-0.422707,-0.456236,-0.825057,-0.992703,-0.992703,-0.322119,-0.883733,-0.473001,-0.422707
5,-0.773853,-0.375001,-0.361594,-0.210768,0.288635,-0.726929,-0.576103,1.160076,0.442813,0.305393,-0.456236,-0.825057,-0.992703,-0.992703,-0.322119,-0.883733,-0.473001,-0.422707,-0.473001,-0.531677
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5595,1.451674,1.220407,0.556771,1.709754,0.483033,0.533309,1.448322,1.820360,1.662831,1.528763,0.759197,0.884931,0.298171,-0.397560,-0.540059,-0.498148,-0.498148,-0.405942,0.164054,0.331700
5596,0.556771,1.709754,0.483033,0.533309,1.448322,1.820360,1.662831,1.528763,1.284089,0.070775,0.298171,-0.397560,-0.540059,-0.498148,-0.498148,-0.405942,0.164054,0.331700,1.052577,1.220223
5597,0.483033,0.533309,1.448322,1.820360,1.662831,1.528763,1.284089,0.070775,0.888589,1.545521,-0.540059,-0.498148,-0.498148,-0.405942,0.164054,0.331700,1.052577,1.220223,1.446545,2.670360


# Timestamp Graph



In [ ]:
def timestamp_graph(df):
  df_plotly_row_100 = pd.DataFrame(df.iloc[100]).T
  df_plotly_row_101 = pd.DataFrame(df.iloc[101]).T
  df_plotly_row_102 = pd.DataFrame(df.iloc[102]).T

  # row 100
  t_slp_df_row_100 = pd.DataFrame(df_plotly_row_100['t_slp'])
  t_surge1_input_df_row_100 = pd.DataFrame(df_plotly_row_100['t_surge1_input'])
  t_surge2_input_df_row_100 = pd.DataFrame(df_plotly_row_100['t_surge2_input'])
  t_surge1_output_df_row_100 = pd.DataFrame(df_plotly_row_100['t_surge1_output'])
  t_surge2_output_df_row_100 = pd.DataFrame(df_plotly_row_100['t_surge2_output'])

  t_slp_df_row_100 = t_slp_df_row_100.explode('t_slp').reset_index(drop=True)
  t_surge1_input_df_row_100 = t_surge1_input_df_row_100.explode('t_surge1_input').reset_index(drop=True)
  t_surge2_input_df_row_100 = t_surge2_input_df_row_100.explode('t_surge2_input').reset_index(drop=True)
  t_surge1_output_df_row_100 = t_surge1_output_df_row_100.explode('t_surge1_output').reset_index(drop=True)
  t_surge2_output_df_row_100 = t_surge2_output_df_row_100.explode('t_surge2_output').reset_index(drop=True)

  # row 101
  t_slp_df_row_101 = pd.DataFrame(df_plotly_row_101['t_slp'])
  t_surge1_input_df_row_101 = pd.DataFrame(df_plotly_row_101['t_surge1_input'])
  t_surge2_input_df_row_101 = pd.DataFrame(df_plotly_row_101['t_surge2_input'])
  t_surge1_output_df_row_101 = pd.DataFrame(df_plotly_row_101['t_surge1_output'])
  t_surge2_output_df_row_101 = pd.DataFrame(df_plotly_row_101['t_surge2_output'])

  t_slp_df_row_101 = t_slp_df_row_101.explode('t_slp').reset_index(drop=True)
  t_surge1_input_df_row_101 = t_surge1_input_df_row_101.explode('t_surge1_input').reset_index(drop=True)
  t_surge2_input_df_row_101 = t_surge2_input_df_row_101.explode('t_surge2_input').reset_index(drop=True)
  t_surge1_output_df_row_101 = t_surge1_output_df_row_101.explode('t_surge1_output').reset_index(drop=True)
  t_surge2_output_df_row_101 = t_surge2_output_df_row_101.explode('t_surge2_output').reset_index(drop=True)

  # row 102
  t_slp_df_row_102 = pd.DataFrame(df_plotly_row_102['t_slp'])
  t_surge1_input_df_row_102 = pd.DataFrame(df_plotly_row_102['t_surge1_input'])
  t_surge2_input_df_row_102 = pd.DataFrame(df_plotly_row_102['t_surge2_input'])
  t_surge1_output_df_row_102 = pd.DataFrame(df_plotly_row_102['t_surge1_output'])
  t_surge2_output_df_row_102 = pd.DataFrame(df_plotly_row_102['t_surge2_output'])

  t_slp_df_row_102 = t_slp_df_row_102.explode('t_slp').reset_index(drop=True)
  t_surge1_input_df_row_102 = t_surge1_input_df_row_102.explode('t_surge1_input').reset_index(drop=True)
  t_surge2_input_df_row_102 = t_surge2_input_df_row_102.explode('t_surge2_input').reset_index(drop=True)
  t_surge1_output_df_row_102 = t_surge1_output_df_row_102.explode('t_surge1_output').reset_index(drop=True)
  t_surge2_output_df_row_102 = t_surge2_output_df_row_102.explode('t_surge2_output').reset_index(drop=True)

  colorvar = '#76878f'

  x = 15
  source_row100 = ColumnDataSource(data=dict(x=pd.to_datetime(t_slp_df_row_100['t_slp']), y=[x] * len(t_slp_df_row_100), size=[10] * len(t_slp_df_row_100),color=[colorvar] * len(t_slp_df_row_100)))
  source2_row100 = ColumnDataSource(data=dict(x=pd.to_datetime(t_surge1_input_df_row_100['t_surge1_input']), y=[x+2] * len(t_surge1_input_df_row_100), size=[10] * len(t_surge1_input_df_row_100),color=[colorvar] * len(t_surge1_input_df_row_100)))
  source3_row100 = ColumnDataSource(data=dict(x=pd.to_datetime(t_surge2_input_df_row_100['t_surge2_input']), y=[x+3] * len(t_surge2_input_df_row_100), size=[10] * len(t_surge2_input_df_row_100),color=[colorvar] * len(t_surge2_input_df_row_100)))
  source2_pred_row100 = ColumnDataSource(data=dict(x=pd.to_datetime(t_surge1_output_df_row_100['t_surge1_output']), y=[x+2] * len(t_surge1_output_df_row_100), size=[10] * len(t_surge1_output_df_row_100),color=['lightgray'] * len(t_surge2_output_df_row_100)))
  source3_pred_row100 = ColumnDataSource(data=dict(x=pd.to_datetime(t_surge2_output_df_row_100['t_surge2_output']), y=[x+3] * len(t_surge2_output_df_row_100), size=[10] * len(t_surge2_output_df_row_100),color=['lightgray'] * len(t_surge2_output_df_row_100)))

  x2 = 8
  source_row101 = ColumnDataSource(data=dict(x=pd.to_datetime(t_slp_df_row_101['t_slp']), y=[x2] * len(t_slp_df_row_101), size=[10] * len(t_slp_df_row_101),color=[colorvar] * len(t_slp_df_row_101)))
  source2_row101 = ColumnDataSource(data=dict(x=pd.to_datetime(t_surge1_input_df_row_101['t_surge1_input']), y=[x2+2] * len(t_surge1_input_df_row_101), size=[10] * len(t_surge1_input_df_row_101),color=[colorvar] * len(t_surge1_input_df_row_101)))
  source3_row101 = ColumnDataSource(data=dict(x=pd.to_datetime(t_surge2_input_df_row_101['t_surge2_input']), y=[x2+3] * len(t_surge2_input_df_row_101), size=[10] * len(t_surge2_input_df_row_101),color=[colorvar] * len(t_surge2_input_df_row_101)))
  source2_pred_row101 = ColumnDataSource(data=dict(x=pd.to_datetime(t_surge1_output_df_row_101['t_surge1_output']), y=[x2+2] * len(t_surge1_output_df_row_101), size=[10] * len(t_surge1_output_df_row_101),color=['lightgray'] * len(t_surge2_output_df_row_101)))
  source3_pred_row101 = ColumnDataSource(data=dict(x=pd.to_datetime(t_surge2_output_df_row_101['t_surge2_output']), y=[x2+3] * len(t_surge2_output_df_row_101), size=[10] * len(t_surge2_output_df_row_101),color=['lightgray'] * len(t_surge2_output_df_row_101)))

  x3 = 1
  source_row102 = ColumnDataSource(data=dict(x=pd.to_datetime(t_slp_df_row_102['t_slp']), y=[x3] * len(t_slp_df_row_102), size=[10] * len(t_slp_df_row_102),color=[colorvar] * len(t_slp_df_row_102)))
  source2_row102 = ColumnDataSource(data=dict(x=pd.to_datetime(t_surge1_input_df_row_102['t_surge1_input']), y=[x3+2] * len(t_surge1_input_df_row_102), size=[10] * len(t_surge1_input_df_row_102),color=[colorvar] * len(t_surge1_input_df_row_102)))
  source3_row102 = ColumnDataSource(data=dict(x=pd.to_datetime(t_surge2_input_df_row_102['t_surge2_input']), y=[x3+3] * len(t_surge2_input_df_row_102), size=[10] * len(t_surge2_input_df_row_102),color=[colorvar] * len(t_surge2_input_df_row_102)))
  source2_pred_row102 = ColumnDataSource(data=dict(x=pd.to_datetime(t_surge1_output_df_row_102['t_surge1_output']), y=[x3+2] * len(t_surge1_output_df_row_102), size=[10] * len(t_surge1_output_df_row_102),color=['lightgray'] * len(t_surge2_output_df_row_102)))
  source3_pred_row102 = ColumnDataSource(data=dict(x=pd.to_datetime(t_surge2_output_df_row_102['t_surge2_output']), y=[x3+3] * len(t_surge2_output_df_row_102), size=[10] * len(t_surge2_output_df_row_102),color=['lightgray'] * len(t_surge2_output_df_row_102)))

  p = figure(plot_width=800, plot_height=400, x_axis_label='Timestamp', y_axis_label='', toolbar_location=None, x_axis_type='datetime')

  y_ticks = [1,2.5,3,4,8,9.5,10,11,15,16.5,17,18]
  y_labels = ['Encoded Image Recorded','Row 102                                                               ','Surge Time, City 1','Surge Time, City 2',
              'Encoded Image Recorded','Row 101                                                               ','Surge Time, City 1','Surge Time, City 2',
              'Encoded Image Recorded','Row 100                                                               ','Surge Time, City 1','Surge Time, City 2']
  p.yaxis.ticker = FixedTicker(ticks=y_ticks)
  p.yaxis.major_label_overrides = {tick: label for tick, label in zip(y_ticks, y_labels)}

  p.ygrid.grid_line_color = 'white'

  sources = [
      source_row100, source2_row100, source3_row100, source2_pred_row100, source3_pred_row100,
      source_row101, source2_row101, source3_row101, source2_pred_row101, source3_pred_row101,
      source_row102, source2_row102, source3_row102, source2_pred_row102, source3_pred_row102
  ]

  for source in sources:
      p.circle('x', 'y', source=source, size='size', alpha=0.5,color='color')

  p.x_range=Range1d(start=pd.Timestamp('1950-11-06 12:30:00'), end=pd.Timestamp('1950-11-18 12:30:00'))

  p.y_range=Range1d(start=0, end=19)

  p.xaxis.major_tick_line_color = None
  p.xaxis.minor_tick_line_color = None
  p.yaxis.major_tick_line_color = None
  p.yaxis.minor_tick_line_color = None

  hover = HoverTool(
      tooltips=[
          ("Time", "@x{%F %T}")
      ],
      formatters={"@x": "datetime"},
      mode='mouse'
  )
  p.add_tools(hover)
  output_notebook()
  show(p)

In [ ]:
timestamp_graph(df_X_train)

The plot above represents three typical rows for the X training dataset. Each row represents a set number of days but will overlap with other rows.

In [ ]:
script, div = components(p)
print(div)
print(script)

# Encoded Image GIF

In [ ]:
def gif(df,column):
  # Retrieve the 40 images from the row of interest
  images = df[column][5599]

  # Create an empty list to store the image file names
  image_files = []

  # Loop over each image and save it as a file
  for i in range(len(images)):
      # Create the file name with a 4-digit index
      filename = f'image_{i:04d}.png'

      # Save the image as a file
      plt.imsave(filename, images[i], cmap='gray')

      # Add the file name to the list
      image_files.append(filename)

  # Use imageio to create the gif
  filename = f'x_train_{column}.gif'

  with imageio.get_writer(filename, mode='I') as writer:
      # Loop over each image file and add it to the gif
      for filename in image_files:
          image = imageio.imread(filename)
          writer.append_data(image)

In [ ]:
gif(df_X_train,'slp')